In [26]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy import stats

def statistici(file_path, sheet1_name, sheet2_name):
   # Citirea tabelului de emisii
    df_emissions = pd.read_excel(file_path, sheet_name=sheet1_name, index_col=0)
    df_emissions.index = df_emissions.index.str.strip()  # Eliminarea spațiilor suplimentare din numele țării

    # Citirea tabelului de cheltuieli
    df_expenditure = pd.read_excel(file_path, sheet_name=sheet2_name, index_col=0)
    df_expenditure.index = df_expenditure.index.str.strip()  # Eliminarea spațiilor suplimentare din numele țării


    # Înlocuim valorile ":" cu numpy.nan în tabelul de cheltuieli
    df_expenditure.replace(":", np.nan, inplace=True)
    df_emissions.replace(":", np.nan, inplace=True)
    

    # Gestionăm valorile lipsă în tabelul de cheltuieli, înlocuind NaN cu cea mai apropiată valoare din coloana respectivă (backward fill)
    df_expenditure = df_expenditure.fillna(method='bfill')
    df_emissions = df_emissions.fillna(method='bfill')
    

    # Conversia valorilor în float, astfel încât să putem calcula corelația
    df_expenditure = df_expenditure.astype(float)
    df_emissions = df_emissions.astype(float)
    

    # Identificăm corelațiile semnificative și coeficienții de determinare pentru fiecare țară
    significant_correlations = {}

    for country in df_emissions.index:
        X = df_expenditure.loc[country].values
        y = df_emissions.loc[country].values
        # Verificăm dacă cele două variabile sunt constante (toate valorile sunt aceleași)
        if np.all(X == X[0]) or np.all(y == y[0]):
            r_squared = 0  # Setăm coeficientul de determinare la 0 în acest caz
            p_value = 1    # Setăm p-value la 1 (nu este semnificativ)
        else:
            model = LinearRegression()
            model.fit(X.reshape(-1, 1), y)
            r_squared = model.score(X.reshape(-1, 1), y)
            _, p_value = stats.pearsonr(X, y)

        significant_correlations[country] = {"r_squared": r_squared, "p_value": p_value}


    # Sortăm țările în funcție de coeficientul de determinare (R^2) în ordine descrescătoare
    sorted_countries = sorted(significant_correlations.items(), key=lambda x: x[1]['p_value'])

    # Identificăm primele trei țări cu cele mai mari coeficienți de determinare și p-valori mici
    top_3_countries = []
    for country, values in sorted_countries:
        if values['r_squared'] > 0.4 and values['p_value'] < 0.05:
            top_3_countries.append((country, values))
            if len(top_3_countries) == 3:
                break

    # Identificăm ultimele trei țări cu cele mai mici coeficienți de determinare și p-valori mari
    bottom_3_countries = []
    for country, values in reversed(sorted_countries):
        bottom_3_countries.append((country, values))
        if len(bottom_3_countries) == 3:
            break

    # Identificăm poziția României în clasament
    rank_of_romania = next((i + 1 for i, (country, _) in enumerate(sorted_countries) if country == 'Romania'), None)

    # Calculam numarul de tari pentru a afisa clasamentul
    num_of_countries = df_emissions.shape[0]

    print("Primele trei țări cu cele mai mari coeficienți de determinare și corelatie mare:")
    for i, (country, values) in enumerate(top_3_countries):
        print(f"{i + 1}. {country}: Corelatie: {1 - values['p_value']:.4f}, Coeficient de determinare (R^2): {values['r_squared']:.2f}")

    print("\nUltimele trei țări cu cele mai mici coeficienți de determinare și corelatie mica:")
    for i, (country, values) in enumerate(reversed(bottom_3_countries)):
        print(f"{num_of_countries - 2 + i}. {country}: Corelatie: {1 - values['p_value']:.4f}, Coeficient de determinare (R^2): {values['r_squared']:.2f}")

    # Afișăm poziția României în clasament
    print(f"\nRomânia se află pe poziția {rank_of_romania} în clasament.")


### Corelatie Emisii/cheltuieli de protectie a mediului

In [72]:
# Încarcăm fișierul Excel
file_path = r'C:\Users\Ricardo Milos\Documents\practica\Date 2012-2019.xlsx'  # Schimbă calea cu locația reală a fișierului
sheet1_name = 'Net greenhouse gas emissions 19'
sheet2_name = 'National expenditure on environ'

statistici (file_path, sheet1_name, sheet2_name)

Primele trei țări cu cele mai mari coeficienți de determinare și corelatie mare:
1. Germany : Corelatie: 0.9961, Coeficient de determinare (R^2): 0.78
2. Czechia: Corelatie: 0.9694, Coeficient de determinare (R^2): 0.57
3. Denmark: Corelatie: 0.9624, Coeficient de determinare (R^2): 0.54

Ultimele trei țări cu cele mai mici coeficienți de determinare și corelatie mica:
26. Cyprus: Corelatie: 0.0992, Coeficient de determinare (R^2): 0.00
27. Luxembourg: Corelatie: 0.0513, Coeficient de determinare (R^2): 0.00
28. Greece: Corelatie: 0.0021, Coeficient de determinare (R^2): 0.00

România se află pe poziția 16 în clasament.


### Corelatie Emisii/PIB

In [73]:
# Încarcăm fișierul Excel
file_path = r'C:\Users\Ricardo Milos\Documents\practica\Date 2012-2019.xlsx'  # Schimbă calea cu locația reală a fișierului
sheet1_name = 'Net greenhouse gas emissions 19'
sheet2_name = 'GDP million euros'

statistici (file_path, sheet1_name, sheet2_name)

Primele trei țări cu cele mai mari coeficienți de determinare și corelatie mare:
1. Lithuania: Corelatie: 0.9994, Coeficient de determinare (R^2): 0.88
2. Czechia: Corelatie: 0.9959, Coeficient de determinare (R^2): 0.77
3. Hungary: Corelatie: 0.9952, Coeficient de determinare (R^2): 0.76

Ultimele trei țări cu cele mai mici coeficienți de determinare și corelatie mica:
26. Latvia: Corelatie: 0.1771, Coeficient de determinare (R^2): 0.01
27. Luxembourg: Corelatie: 0.1757, Coeficient de determinare (R^2): 0.01
28. Bulgaria: Corelatie: 0.0399, Coeficient de determinare (R^2): 0.00

România se află pe poziția 25 în clasament.


In [27]:
# Încarcăm fișierul Excel
file_path = r'C:\Users\Ricardo Milos\Documents\practica\Toate periate.xlsx'  # Schimbă calea cu locația reală a fișierului
sheet1_name = 'electric locomotives'
sheet2_name = 'Net greenhouse gas emissions 19'


statistici (file_path, sheet1_name, sheet2_name)

Primele trei țări cu cele mai mari coeficienți de determinare și corelatie mare:
1. Poland: Corelatie: 0.9993, Coeficient de determinare (R^2): 0.87
2. Czechia: Corelatie: 0.9989, Coeficient de determinare (R^2): 0.85
3. Belgium: Corelatie: 0.9563, Coeficient de determinare (R^2): 0.52

Ultimele trei țări cu cele mai mici coeficienți de determinare și corelatie mica:
23. Lithuania: Corelatie: 0.0000, Coeficient de determinare (R^2): 0.00
24. Luxembourg: Corelatie: 0.0000, Coeficient de determinare (R^2): 0.00
25. Slovenia: Corelatie: 0.0000, Coeficient de determinare (R^2): 0.00

România se află pe poziția 20 în clasament.
